In [ ]:
import torch

In [ ]:
print(torch.__version__)

In [ ]:
arr_org = torch.tensor([[1, 2, 3], [4, 5, 6]])
print(hex(id(arr_org)))
print(hex(arr_org.data_ptr()))

In [ ]:
arr1 = arr_org.reshape(3, 2)
print(hex(id(arr1)))
print(hex(arr1.data_ptr()))

In [ ]:
arr2 = arr_org.reshape(1, 6)
print(hex(id(arr2)))
print(hex(arr2.data_ptr()))

In [ ]:
%matplotlib inline
from d2l import torch as d2l
import random
from typing import Tuple

# 定义随机数据

In [ ]:
def synthetic_data(w: torch.Tensor, b: float, num_examples: int) -> Tuple[torch.Tensor, torch.Tensor]:
    X = torch.normal(0, 1, (num_examples, w.shape[0]))
    y = torch.matmul(X, w) + b
    y += torch.normal(0, 0.01, y.shape)
    return X, y

In [ ]:
true_w = torch.tensor([2, -3.4])
true_b = 4.3
features, labels = synthetic_data(true_w, true_b, 1000)
print(features.shape, labels.shape)

In [ ]:
d2l.set_figsize((8,4))
d2l.plt.scatter(features[:, 1], labels, 1)

# 定义数据迭代器

In [ ]:
def data_iter(batch_size, features, labels):
    num_examples = len(features)
    indices = list(range(num_examples))
    random.shuffle(indices)
    for i in range(0, num_examples, batch_size):
        batch_indices = torch.tensor(indices[i: min(i + batch_size, num_examples)])
        yield features[batch_indices], labels[batch_indices]

In [ ]:
batch_size = 10
for X, y in data_iter(batch_size, features, labels):
    print(X, '\n', y)
    break

# 初始化数据

In [ ]:
w = torch.normal(0, 0.01, size=(2, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)
print(w.shape, b.shape)

# 定义线性模型

In [ ]:
def linreg(X: torch.Tensor, w: torch.Tensor, b: torch.Tensor) -> torch.Tensor:
    return torch.matmul(X, w) + b

In [ ]:
print(linreg(features, w, b).shape)

# 定义损失函数

In [ ]:
def squared_loss(y_hat: torch.Tensor, y: torch.Tensor) -> torch.Tensor:
    return (y_hat - y.reshape(y_hat.shape)) ** 2 / 2

In [ ]:
y_hat = linreg(features, w, b)
loss = squared_loss(y_hat, labels)
print(loss.shape)

# 定义优化算法

In [ ]:
def sgd(params: Tuple[torch.Tensor], lr: float, batch_size: int) -> None:
    with torch.no_grad():
        for param in params:
            param.data -= lr * param.grad / batch_size
            param.grad.zero_()

# 训练模型

In [ ]:
lr = 0.03
num_epochs = 3
net = linreg
loss_fn = squared_loss
batch_size = 10

In [ ]:
for epoch in range(num_epochs):
    for X, y in data_iter(10, features, labels):
        l = loss_fn(net(X, w, b), y)
        l.sum().backward()
        sgd([w, b], lr, batch_size)
    with torch.no_grad():
        train_l = loss_fn(net(features, w, b), labels)
        print(f'epoch {epoch + 1}, loss {float(train_l.mean()):f}')